**QUESTION - 1**

***Problem statement*** - Test drive the basic version of Apriori and FP Growth algorithms for Frequent Itemset Mining using the package / library support in the platform of your choice. Test it with various support and confidence measures and generate a time comparison for varied data set sizes. To do the performance comparison you may use benchmark datasets provided for FIM such as the FIMI workshop or other sources.

***Logic used*** - 

***Packages used*** - 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori
from pandas import DataFrame
from mlxtend.frequent_patterns import apriori

data1 = pd.read_csv("data.csv",header=None)
binary_matrix = [[0 for i in range (118)] for j in range (data1.shape[0])]

for i in range (data1.shape[0]):
  for j in range (data1.shape[1]-1):
    x=int(data1[j][i])
    binary_matrix[i][x]=1

BM = DataFrame(binary_matrix)
apriori(BM, min_support=0.6)

,support,itemsets
0,0.89750,(2)
1,0.81750,(23)
2,1.00000,(34)
3,0.82875,(36)
4,0.72125,(39)
...,...,...
9722,0.62500,"(2, 34, 67, 39, 76, 52, 85, 86, 23, 90, 59, 93)"
9723,0.62500,"(2, 67, 36, 39, 76, 52, 85, 86, 23, 90, 59, 93)"
9724,0.62500,"(2, 34, 36, 67, 39, 76, 52, 85, 86, 90, 59, 93)"
9725,0.62500,"(34, 67, 36, 39, 76, 52, 85, 86, 23, 90, 59, 93)"


**QUESTION - 2**

**Problem statement** - Extend the Apriori Algorithm discussed in the class supporting Transaction Reduction approach to improve the time complexity issue as a result of the repeated scans limitation of Apriori. You may compare this extended version with the earlier implementations in (1) over the same benchmark dataset.

**Logic used** - Implement transaction reduction

**Packages used** -
* from itertools import combinations
* pandas


In [ ]:
from itertools import combinations
import pandas as pd

def aprioriTransactionReduction(transactions, min_support_count, verbose=False):
	if verbose:
		print("Database Transactions")
		for transaction in transactions:
			print(transaction)
	print("Number of Transactions = {}".format(len(transactions)))

	items = set()
	for transaction in transactions:
		items.update(transaction)
	items = sorted(list(items))
	frequent_itemsets = []
	level_k = 1 # The current level number
	level_frequent_itemsets = [] # Level 0: Frequent itemsets
	candidate_frequent_itemsets = [[item] for item in items] # Level 1: Candidate itemsets

	while candidate_frequent_itemsets:
		if verbose:
			print("\n==================================================================")
			print("Level {}: Frequent Itemsets".format(level_k-1))
			for level_frequent_itemset in level_frequent_itemsets:
				print(level_frequent_itemset)
			print()
			print("Level {}: Pruned Transactions".format(level_k))

		candidate_freq_itemsets_cnts = [0]*len(candidate_frequent_itemsets)
		current_trans_index = 0
		for transaction in transactions:
			trans_n_candidate_itemsets = 0
			for i, itemset in enumerate(candidate_frequent_itemsets):
				if all(_item in transaction for _item in itemset):
					candidate_freq_itemsets_cnts[i] += 1
					trans_n_candidate_itemsets += 1
			if trans_n_candidate_itemsets > 1:
				transactions[current_trans_index] = transaction
				current_trans_index += 1
			elif verbose:
				print(transaction)
		if verbose:
			print("Number of transactions pruned = {}".format(len(transactions)-current_trans_index), end='\n\n')
		transactions = transactions[:current_trans_index]
		print("After Level {}: Number of Transactions = {}".format(level_k, current_trans_index))
		if verbose:
			print("==================================================================")
   
		level_frequent_itemsets = [itemset for itemset, support in zip(candidate_frequent_itemsets, candidate_freq_itemsets_cnts) if support >= min_support_count]
		frequent_itemsets.extend([set(level_frequent_itemset) for level_frequent_itemset in level_frequent_itemsets])
		candidate_frequent_itemsets = generateCandidateItemsets(level_k, level_frequent_itemsets)
		level_k += 1
	return frequent_itemsets


def generateCandidateItemsets(level_k, level_frequent_itemsets):
	n_frequent_itemsets = len(level_frequent_itemsets)
	candidate_frequent_itemsets = []
	for i in range(n_frequent_itemsets):
		j = i+1
		while (j<n_frequent_itemsets) and (level_frequent_itemsets[i][:level_k-1] == level_frequent_itemsets[j][:level_k-1]):
			candidate_itemset = level_frequent_itemsets[i][:level_k-1] + [level_frequent_itemsets[i][level_k-1]] + [level_frequent_itemsets[j][level_k-1]]
			candidate_itemset_pass = False
			if level_k == 1:
				candidate_itemset_pass = True
			elif (level_k == 2) and (candidate_itemset[-2:] in level_frequent_itemsets):
				candidate_itemset_pass = True
			elif all((list(_)+candidate_itemset[-2:]) in level_frequent_itemsets for _ in combinations(candidate_itemset[:-2], level_k-2)):
				candidate_itemset_pass = True				
			if candidate_itemset_pass:
				candidate_frequent_itemsets.append(candidate_itemset)
			j += 1
	return candidate_frequent_itemsets


if __name__ == '__main__':
  data2 = pd.read_csv("data3.csv",header=None)
  data2.drop(data2.columns[len(data2.columns)-1], axis=1, inplace=True)
  transactions=data2.values.tolist()
  min_support_count = 3
  verbose = True
  frequent_itemsets = aprioriTransactionReduction(transactions, min_support_count, verbose=verbose)
  print("\nFREQUENT ITEMSETS (Min Support Count = {})".format(min_support_count))
  for frequent_itemset in frequent_itemsets:
    print(frequent_itemset)

Streaming output truncated to the last 5000 lines.
[14, 59, 67, 86]
[14, 59, 67, 90]
[14, 59, 67, 93]
[14, 59, 76, 85]
[14, 59, 76, 86]
[14, 59, 76, 90]
[14, 59, 76, 93]
[14, 59, 85, 86]
[14, 59, 85, 90]
[14, 59, 85, 93]
[14, 59, 86, 90]
[14, 59, 86, 93]
[14, 59, 90, 93]
[14, 63, 67, 76]
[14, 63, 67, 85]
[14, 63, 67, 86]
[14, 63, 67, 90]
[14, 63, 67, 93]
[14, 63, 76, 85]
[14, 63, 76, 86]
[14, 63, 76, 90]
[14, 63, 76, 93]
[14, 63, 85, 86]
[14, 63, 85, 90]
[14, 63, 85, 93]
[14, 63, 86, 90]
[14, 63, 86, 93]
[14, 63, 90, 93]
[14, 67, 76, 85]
[14, 67, 76, 86]
[14, 67, 76, 90]
[14, 67, 76, 93]
[14, 67, 85, 86]
[14, 67, 85, 90]
[14, 67, 85, 93]
[14, 67, 86, 90]
[14, 67, 86, 93]
[14, 67, 90, 93]
[14, 76, 85, 86]
[14, 76, 85, 90]
[14, 76, 85, 93]
[14, 76, 86, 90]
[14, 76, 86, 93]
[14, 76, 90, 93]
[14, 85, 86, 90]
[14, 85, 86, 93]
[14, 85, 90, 93]
[14, 86, 90, 93]
[15, 23, 34, 36]
[15, 23, 34, 39]
[15, 23, 34, 41]
[15, 23, 34, 52]
[15, 23, 34, 55]
[15, 23, 34, 59]
[15, 23, 34, 63]
[15, 23, 34, 6

KeyboardInterrupt: ignored

**QUESTION - 3**

**Problem statement** - Test drive any one implementation in (1) or (2) adopting a Vertical Transaction Database format.

**Logic used** - Year > 2016 then 1, else 0.

**Packages used** -
* 
* 


**QUESTION - 4**

**Problem statement** - Using a vertical transaction database notation, generate the FI’s following the intersection approach (basic ECLAT) discussed in the class. Use earlier benchmark datasets in (1).

**Logic used** - The basic ECLAT algo is implemented using the vertical database.

**Packages used** -
* from fim import apriori, eclat, fpgrowth, fim
* pandas

In [ ]:
from fim import apriori, eclat, fpgrowth, fim
import pandas as pd

data4 = pd.read_csv("eclat4.csv", header=None)
trans4=data4.values.tolist()

print('transactions:')
for t in trans4:
  print(t)

print  ('\nEclat supp=3, min_items_per_set=2')
for r in eclat(trans4, supp=-1, zmin=2):
  print(r)

transactions:
[1, 2, 3]
[1, 4, 5]
[2, 3, 4]
[1, 2, 3]
[4, 2, 3]
[1, 2, 4]
[3, 4, 5]
[1, 2, 3]

Eclat supp=3, min_items_per_set=2
((3, 2), 5)
((1, 2), 4)
((1, 3, 2), 3)
((1, 3), 3)
((4, 2), 3)
((4, 3, 2), 2)
((4, 3), 3)
((4, 1, 2), 1)
((4, 1), 2)
((5, 3, 4), 1)
((5, 3), 1)
((5, 1, 4), 1)
((5, 1), 1)
((5, 4), 2)


**QUESTION - 5**

**Problem statement** - Extend the basic Apriori algorithm to generate Frequent Patterns which differentiate ab from ba (ordered patterns generation).

**Logic used** - Instead of considering ab and ba as one, we treat them as different items and calculate the support counts

**Packages used** - None

In [ ]:
trans5 = [['a','b','c'],['a','c','d'],['c','b','d'],['d','a','c'],['d','a','c']]
min_sup = 3
count_1 = [0,0,0,0,0]
count_2 = [0,0,0,0,0,0]
C1 = ['a','b','c','d']
L1 = []
C2 = []
L2 = []

for j in range (0, len(trans5)):
  for i in range (0, 3):
    if(trans5[j][i] == 'a'):
      count_1[1]=count_1[1]+1
    if(trans5[j][i] == 'b'):
      count_1[2]=count_1[2]+1
    if(trans5[j][i] == 'c'):
      count_1[3]=count_1[3]+1
    if(trans5[j][i] == 'd'):
      count_1[4]=count_1[4]+1
print("\nC1: ")
print(C1)
print(count_1[1:5])

print("\nL1: ")
for j in range (1,5):
  if(count_1[j] >=min_sup ):
    print(chr(96+j), end = ' ')
    L1.append(chr(96+j))
print()

print("\nC2: ")
for i in range (0,len(L1)):
  for j in range (0,len(L1)):
    x1=L1[i]
    x2=L1[j]
    if(x1 != x2):
      x3=x1+x2
      C2.append(x3)
print(C2)

for i in range (0,len(C2)):
  str1=C2[i]
  for k in range (0,len(trans5)):
    d1=0
    d2=0
    for j in range (0,3):
      if(str1[0] == trans5[k][j]):
        d1=1
        for l in range (j,3):
          if(str1[1] == trans5[k][l]):
            d2=1
      if(d1==1 and d2==1):
        count_2[i]=count_2[i]+1
        break
print(count_2)

print('\nL2')
for i in range (len(C2)):
  if(count_2[i]>=min_sup):
    L2.append(C2[i])
print(L2)


C1: 
['a', 'b', 'c', 'd']
[4, 2, 5, 4]

L1: 
a c d 

C2: 
['ac', 'ad', 'ca', 'cd', 'da', 'dc']
[4, 1, 0, 2, 2, 2]

L2
['ac']


**QUESTION - 6**

**Problem statement** - Implement following extensions to Apriori Algorithm (discussed / to be discussed in the class): Hash based strategy, Partitioning Approach and Sampling strategies. You may refer to online tutorials for a formal pseudocode description.

**Logic used** - 
* Hash based approach
* Partitioning approach

**Packages used** - 
* itertools
* time

In [ ]:
import itertools
import time

filename ="data3_ws"
min_support = 5

def frequent_one_item(Transaction,min_support):
    candidate1 = {}
    for i in range(0,len(Transaction)):
        for j in range(0,len(Transaction[i])):
            if Transaction[i][j] not in candidate1:
                candidate1[Transaction[i][j]] = 1
            else:
                candidate1[Transaction[i][j]] += 1
    frequentitem1 = []                      
    for value in candidate1:
        if candidate1[value] >= min_support:
            frequentitem1 = frequentitem1 + [[value]]
            Frequent_items_value[tuple(value)] = candidate1[value]
    return frequentitem1


class Hash_node:
    def __init__(self):
        self.children = {}           #pointer to its children
        self.Leaf_status = True      #to know the status whether current node is leaf or not
        self.bucket = {}             #contains itemsets in bucket

#class of constructing and getting hashtree
class HashTree:
    # class constructor
    def __init__(self, max_leaf_count, max_child_count):
        self.root = Hash_node()
        self.max_leaf_count = max_leaf_count
        self.max_child_count = max_child_count
        self.frequent_itemsets = []

    # function to recursive insertion to make hashtree
    def recursively_insert(self, node, itemset, index, count):
        if index == len(itemset):
            if itemset in node.bucket:
                node.bucket[itemset] += count
            else:
                node.bucket[itemset] = count
            return

        if node.Leaf_status:                             #if node is leaf
            if itemset in node.bucket:
                node.bucket[itemset] += count
            else:
                node.bucket[itemset] = count
            if len(node.bucket) == self.max_leaf_count:  #if bucket capacity increases
                for old_itemset, old_count in node.bucket.items():

                    hash_key = self.hash_function(old_itemset[index])  #do hashing on next index
                    if hash_key not in node.children:
                        node.children[hash_key] = Hash_node()
                    self.recursively_insert(node.children[hash_key], old_itemset, index + 1, old_count)
                #since no more requirement of this bucket
                del node.bucket
                node.Leaf_status = False
        else:                                            #if node is not leaf
            hash_key = self.hash_function(itemset[index])
            if hash_key not in node.children:
                node.children[hash_key] = Hash_node()
            self.recursively_insert(node.children[hash_key], itemset, index + 1, count)

    def insert(self, itemset):
        itemset = tuple(itemset)
        self.recursively_insert(self.root, itemset, 0, 0)

    # to add support to candidate itemsets. Transverse the Tree and find the bucket in which this itemset is present.
    def add_support(self, itemset):
        Transverse_HNode = self.root
        itemset = tuple(itemset)
        index = 0
        while True:
            if Transverse_HNode.Leaf_status:
                if itemset in Transverse_HNode.bucket:    #found the itemset in this bucket
                    Transverse_HNode.bucket[itemset] += 1 #increment the count of this itemset.
                break
            hash_key = self.hash_function(itemset[index])
            if hash_key in Transverse_HNode.children:
                Transverse_HNode = Transverse_HNode.children[hash_key]
            else:
                break
            index += 1

    # to transverse the hashtree to get frequent itemsets with minimum support count
    def get_frequent_itemsets(self, node, support_count,frequent_itemsets):
        if node.Leaf_status:
            for key, value in node.bucket.items():
                if value >= support_count:                       #if it satisfies the condition
                    frequent_itemsets.append(list(key))          #then add it to frequent itemsets.
                    Frequent_items_value[key] = value
            return

        for child in node.children.values():
            self.get_frequent_itemsets(child, support_count,frequent_itemsets)

    # hash function for making HashTree
    def hash_function(self, val):
        return int(val) % self.max_child_count


#To generate hash tree from candidate itemsets
def generate_hash_tree(candidate_itemsets, max_leaf_count, max_child_count):
    htree = HashTree(max_child_count, max_leaf_count)             #create instance of HashTree
    for itemset in candidate_itemsets:
        htree.insert(itemset)                                     #to insert itemset into Hashtree
    return htree


#to generate subsets of itemsets of size k
def generate_k_subsets(dataset, length):
    subsets = []
    for itemset in dataset:
        subsets.extend(map(list, itertools.combinations(itemset, length)))
    return subsets


def subset_generation(ck_data,l):
    return map(list,set(itertools.combinations(ck_data,l)))


#apriori generate function to generate ck
def apriori_generate(dataset,k):
    ck = []
    #join step
    lenlk = len(dataset)
    for i in range(lenlk):
        for j in range(i+1,lenlk):
            L1 = list(dataset[i])[:k - 2]
            L2 = list(dataset[j])[:k - 2]
            if L1 == L2:
                ck.append(sorted(list(set(dataset[i]) | set(dataset[j]))))

    #prune step
    final_ck = []
    for candidate in ck:
        all_subsets = list(subset_generation(set(candidate), k - 1))
        found = True
        for i in range(len(all_subsets)):
            value = list(sorted(all_subsets[i]))
            if value not in dataset:
                found = False
        if found == True:
            final_ck.append(candidate)

    return ck,final_ck


def generateL(ck,min_support):
    support_ck = {}
    for val in Transaction1:
        for val1 in ck:
            value = set(val)
            value1 = set(val1)

            if value1.issubset(value):
                if tuple(val1) not in support_ck:
                    support_ck[tuple(val1)] = 1
                else:
                    support_ck[tuple(val1)] += 1
    frequent_item = []
    for item_set in support_ck:
        if support_ck[item_set] >= min_support:
            frequent_item.append(sorted(list(item_set)))
            Frequent_items_value[item_set] = support_ck[item_set]

    return frequent_item


# main apriori algorithm function
def apriori(L1,min_support):
    k = 2;
    L = []
    L.append(0)
    L.append(L1)
    print("enter max_leaf_count")              #maximum number of items in bucket i.e. bucket capacity of each node
    max_leaf_count = 25#int(input())
    print("enter max_child_count")             #maximum number of child you want for a node
    max_child_count = 5#int(input())

    start = time.time()
    while(len(L[k-1])>0):
        ck,final_ck = apriori_generate(L[k-1],k)                 #to generate candidate itemsets
        print("C%d" %(k))
        print(final_ck)
        h_tree = generate_hash_tree(ck,max_leaf_count,max_child_count)       #to generate hashtree
        if (k > 2):
            while(len(L[k-1])>0):
                l = generateL(final_ck, min_support)
                L.append(l)
                print("Frequent %d item" % (k))
                print(l)
                k = k + 1
                ck, final_ck = apriori_generate(L[k - 1], k)
                print("C%d" % (k))
                print(final_ck)
            break
        k_subsets = generate_k_subsets(Transaction1,k)                  #to generate subsets of each transaction
        for subset in k_subsets:
            h_tree.add_support(subset)                                  #to add support count to itemsets in hashtree
        lk = []
        h_tree.get_frequent_itemsets(h_tree.root,min_support,lk)                  #to get frequent itemsets
        print("Frequent %d item" %(k))
        print(lk)
        L.append(lk)
        k = k + 1
    end = time.time()
    return L,(end-start)


with open(filename) as f:
  content = f.readlines()
content = [x.strip() for x in content]
Transaction = []                  
Frequent_items_value = {}         

for i in range(0,len(content)):
    Transaction.append(content[i].split())

values = frequent_one_item(Transaction,min_support)
print(values)
print(Frequent_items_value)

Transaction1 = []
for i in range(0,len(Transaction)):
    list_val = []
    for j in range(0,len(Transaction[i])):
        if [Transaction[i][j]] in values:
            list_val.append(Transaction[i][j])
    Transaction1.append(list_val)

L_value,time_taken = apriori(values,min_support)
print("All frequent itemsets with their support count:")
print(Frequent_items_value)

In [ ]:
import itertools
from collections import defaultdict
f=open("part.txt","r");
support=[2,1,2,1]
avg=float(sum(support))/4
lines = f.readlines()
lines2d=[['' for x in range(0)] for x in range(4)]

distinct = defaultdict(dict)    
overall_distinct=[]

i=1;
for line in lines:
    line=line.strip()
    if('-' in line):
        i=i+1
        continue
    else:
        lines2d[i-1].append(line)
    for item in line.split(','):
        if(item not in distinct[i].keys()):
            distinct[i][item]=1
        else:
            distinct[i][item]=distinct[i][item]+1
        if(item not in overall_distinct):
            overall_distinct.append(item)

list1 = [['' for x in range(0)] for x in range(4)] 
frequent=[['' for x in range(0)] for x in range(4)] 

print ('------C 1-----')
for i in range(1,5,1):
    for key in sorted(distinct[i]):
            print ("('%s') %d" % (key, distinct[i][key]))
    print ("-----\n")
        
print ('\n******L 1*****')
for i in range(1,5,1):
    for key in sorted(distinct[i]):
        if(distinct[i][key]>=support[i-1]):
            print ("('%s') %d" % (key, distinct[i][key]))
            if(key not in list1):
                list1[i-1].append(key)
                frequent[i-1].append(key)
    print ("-----")

print ("\n\n")

newlist=[['' for x in range(0)] for x in range(4)] 

for i in range(1,5,1):
    print ('\n\nFOR PARTITION NO. : ',i,'\n\n')
    length=len(list1[i-1])
    L=2
    while(L<=length):
        print ('------C',L,'-----')
        for subset in itertools.combinations(list1[i-1], L):
                for line in lines2d[i-1]:
                    line=line.strip().split(',')
                    if(subset not in distinct[i]):
                        distinct[i][subset]=0
                    if(set(subset).issubset(set(line))):
                        distinct[i][subset]=distinct[i][subset]+1
    
                print (subset, distinct[i][subset])
        print ('******L',L,'*****')
        for subset in itertools.combinations(list1[i-1], L):
                if(distinct[i][subset]>=support[i-1]):
                    print (subset, distinct[i][subset])
                    frequent.append(subset)
                    for j in subset:
                        if j not in newlist[i-1]:
                            newlist[i-1].append(j)
        list1[i-1]=newlist[i-1]
        newlist[i-1]=[]
        length=len(list1[i-1])
        print ("\n\n")
        L=L+1
        
new_elements=[]
for l in range(len(overall_distinct)):
    for subset in itertools.combinations(overall_distinct, l):
        total=0        
        count=0
        for i in range(1,5):
            for j in distinct[i].keys():
                if(sorted(subset)==sorted(j) and distinct[i][j]>=support[i-1]):
                    total=total+distinct[i][j]
                    count=count+1
                    break
        #print sorted(subset)
        if(count==0):
            continue
        else:
            print ("('%s') %.2f" % (subset, float(total)/float(count)))
            if(float(total)/float(count)>=avg):
                new_elements.append(subset)

print ('\n\n')

for k in new_elements:
    total=0
    for i in range(1,5):
        for j in distinct[i].keys():
            if(sorted(k)==sorted(j)):
                total=total+distinct[i][j]
                break
    print(sorted(k), (float(total)/float(4)))

**QUESTION - 7**

**Problem statement** - Implement the Dynamic Itemset Counting Algorithm for Frequent Itemset Generation.

**Logic used** - The attributes which have the same number of null attributes as the max number of null values are found manually and dropped.

**Packages used** - 
* numpy
* itertools
* copy

In [ ]:
import numpy as np
import itertools 
import copy
			

def subset_generator(S,n):
	a = itertools.combinations(S,n)
	result = []
	for i in a:
		result.append(set(i))
	return(result)	


def superset_generator(S,unique_itemset):
	result = []
	a = set()
	for i in unique_itemset:
		if i.intersection(S)==set():
			a = i.union(S)
			result.append(a)
			a = set()
	return(result)		


def subset_checker(S,FIS):
	subset = subset_generator(S,len(S)-1)
	flag = 1
	temp = []
	for i in FIS:
		temp.append(i[0])
	FIS = temp
	for i in subset:
		if i not in FIS:
			flag=0
			break
	if flag:
		return(True)
	else:
		return(False)


def transaction_to_itemset(T):
	result = set()
	for i in range(len(T)):
		if T[i]!=0:
			result.add(i+1)
	return(result)		


database = [[1,1,0,0],[1,0,1,0],[0,1,1,1],[0,1,0,0]]
unique_itemset =[{1},{2},{3},{4}]
min_supp = 1
M = 2
size = len(database)

#4 lists are simplemented which store the state if itemsers in the database
DC = []
DS = []	
SC = []
SS = []

DC = [[i,0,0] for i in unique_itemset]
print("Initial DC:",DC,"\n")

counter = 0
T = []
while DC!=[] or DS!=[]:
#for p in range(6):
	for i in range(counter,counter+M):				#updating counter var in each itemset
		index = i%size
		T = transaction_to_itemset(database[index])
		print("Transaction :",T)

		for item in DC:
			item[2]+=1
			if item[0].issubset(T):
				item[1]+=1
		for item in DS:
			item[2]+=1
			if item[0].issubset(T):
				item[1]+=1		
	for item in copy.copy(DC):								#transfer itemsets from DC to DS basef on min_supp
		if(item[1]>=min_supp):
			DS.append(item)
			DC.remove(item)
	for item in copy.copy(DS):								#transfer itemsets from DS to SS if it's count=size
		if(item[2]==size):
			SS.append(item)
			DS.remove(item)			
	for item in copy.copy(DC):								#transfer itemsets from DC to SC if it's count=size
		if(item[2]==size):
			SC.append(item)
			DC.remove(item)
	#print(DC)
	
	FIS = copy.copy(DS)
	FIS.extend(SS)								#check if all subsets are there in FIS for immediate supersets
	for item in FIS:
		S = superset_generator(item[0],unique_itemset)
		for i in S:
			if subset_checker(i,FIS):
				flag=1
				for x in DC:
					if x[0]==i:
						flag=0
				for x in DS:
					if x[0]==i:
						flag=0
				for x in SC:
					if x[0]==i:
						flag=0
				for x in SS:
					if x[0]==i:
						flag=0						
				if flag:
					DC.append([i,0,0])		

	counter+=M
	print("DS: ",DS)
	print("DC: ",DC)
	print("SS: ",SS)
	print("SC: ",SC,"\n")

**QUESTION - 8**

**Problem statement** - Test drive (download exe’s or generate one using open source versions) any three algorithm for FIM not discussed in the class.

**Logic used** - Implemented apriori algorithm using inbuilt package.

**Packages used** - 
* pandas
* from efficient_apriori import apriori

In [ ]:
import pandas as pd
from efficient_apriori import apriori

data81 = pd.read_csv("data3.csv")
trans8=data81.values.tolist()

itemsets, rules = apriori(trans8, min_support=0.9, min_confidence=1)

print('itemsets\n')
print(itemsets)
print('rules\n')
print(rules)

**QUESTION - 9**

**Problem statement** - For any of the real time dataset shared by my TA earlier or some other online challenge dataset, implement pre-processing strategies required and generate associations amongst the attributes of interests using any one FP implementation.

**Logic used** - Nan values replaced with mean of Average price, the xxxx and yyyy values are replaced with 'Albany', and then apriori algorithm is implemented.

**Packages used** - 
* pandas
* numpy
* statistics
* from mlxtend.frequent_patterns import apriori, association_rules
* from apriori_python import apriori

In [ ]:
import pandas as pd
import numpy as np
import statistics
from mlxtend.frequent_patterns import apriori, association_rules
from apriori_python import apriori


ap=[]

data9 = pd.read_csv("trail_small.csv")

#data cleaning 
for i in range (data9.shape[0]):
  y=data9["AveragePrice"][i]
  z=data9["region"][i]
  if(np.isnan(y) == False):
    ap.append(y)
  if(z == "xxxx" or z == "yyyy"):
    data9["region"][i] = "Albany"#replacing xxxx or yyyy to default Albany

#replacing the nulll values with mean in average price
for i in range (data9.shape[0]):
  y=data9["AveragePrice"][i]
  if(np.isnan(y)):
    data9["AveragePrice"][i]=statistics.mean(ap)

data_list = data9.values.tolist()
print('data: \n')
print(data_list)
print()
freqItemSet, rules = apriori(data_list, minSup=0.4,minConf=0.5)
print('\nfreqItemSet\n')
print(freqItemSet)
print('\nrules\n')
print(rules)
print('\n')